# Stochastic Partial Derivative Equations

<!-- SUMMARY: Estimation and Simulations performed in the framework of SPDE -->

<!-- CATEGORY: SPDE -->

In this tutorial, we show how to use the SPDE. We compare some calculations performed *by hand* with the results obtained through gstlearn API interfaces. Note that we also consider (probably temporarily) the Old interface (instantiating the SPDE class and performing subsequent calculations within this class) and the new interface where individual functions are designed for Kriging, Simulating and calculating LogLikelihood.

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import numpy as np
import matplotlib.pyplot as plt

from sksparse.cholmod import cholesky
import scipy as sc
from scipy.sparse import *
from scipy.sparse.linalg import *
import numpy as np

gdoc.setNoScroll()

## Parameters

In [ ]:
# Data
np.random.seed(123)
ndat = 1000

# Model
rangev = 0.2
sill = 1.
nugget = 0.1

# Grid 
nx = [50,50]
dx = [0.02,0.02]
x0 = [0,0]

#Grid meshing
nxm = [75,75]
dxm = [0.02,0.02]
x0m = [-0.25,-0.25]

figsize = [5,5]

dbfmt = gl.DbStringFormat.createFromFlags(flag_stats=True, names=["*SPDE"])

### Grid and Meshing

The 'grid' object defines the grid where API calculations will be performed

In [ ]:
grid = gl.DbGrid.create(nx,dx,x0)

The 'gridExt' object is a dilated field which is used for manual calculation and to establish the Unique Mesh. The selection 'newSel' masks off the cells which are not covering 'grid'. The vector 'indSel' gives the indices of the active cells.

In [ ]:
gridExt = gl.DbGrid.create(nxm,dxm,x0m)

mesh = gl.MeshETurbo(gridExt)
meshes = gl.VectorMeshes()
meshes.push_back(mesh)
gl.dumpMeshes(meshes)

gridExt.addSelection((gridExt["x1"]>0) & (gridExt["x2"]>0) & (gridExt["x1"]<1.) & (gridExt["x2"]<1.))
indSel = np.where(gridExt["NewSel"])[0]

### Model

In [ ]:
model = gl.Model.createFromParam(gl.ECov.MATERN,param=1,range=rangev,sill=sill)
model.addCovFromParam(gl.ECov.NUGGET,sill=nugget)
model.setDriftIRF()

### Data

In [ ]:
dat = gl.Db.create()
dat["x"] = np.random.uniform(size=ndat)
dat["y"] = np.random.uniform(size=ndat)
dat.setLocators(["x","y"],gl.ELoc.X)

## SPDE non-conditional simulation

We perform a non-conditional simulation of the Model on the grid in order to get the graphic rendition of the corresponding texture.

### Performed on Grid

In [ ]:
gl.law_set_random_seed(131351)
gl.simulateSPDE(None, grid, model, 1, -1, meshes)
grid.display(dbfmt)

In [ ]:
gp.init(figsize=figsize,flagEqual=True)
gp.raster(grid)
gp.decoration(title="Non Conditional Simulation")

### Performed on Data

We also perform a non conditional simulation on the Data location, which will serve for future conditioning information (for Kriging and Conditional Simulations).

In [ ]:
gl.law_set_random_seed(131351)
gl.simulateSPDE(None, dat, model, 1, -1, meshes)
dat.display(dbfmt)

## Kriging

We perform an estimation by Kriging in the SPDE framework:

- either using the Matrix Free version

In [ ]:
err = gl.krigingSPDE(dat, gridExt, model, useCholesky=0, meshesK = meshes, namconv=gl.NamingConvention("KrigingSPDE-Free"), verbose=False)

- or using the Matrix version and the Cholesky decomposition. This is the version that will serve for comparion with the manual demonstration of the methodology further. 

In [ ]:
err = gl.krigingSPDE(dat, gridExt, model, useCholesky=1, meshesK = meshes, namconv=gl.NamingConvention("KrigingSPDE-Chol"),verbose=False)

Comparison between the two ways to perform SPDE internal algorithm

In [ ]:
ax = plt.scatter(gridExt["*Chol.*estim"],gridExt["*Free.*estim"],s=1)
p = plt.plot([-3,3],[-3,3],c="r")

In [ ]:
gp.init(figsize=figsize,flagEqual=True)
gp.raster(gridExt)
gp.symbol(dat, c='black')
gp.decoration(title="Estimation")

In [ ]:
gridExt.display(dbfmt)

## Manually

All calculations are performed on 'gridExt'... but the comparison with the API is performed on its restriction to 'grid'.

In [ ]:
spdeRes = gl.SPDE(dat, model, True, False, 1)
spdeRes.setMeshes(True, meshes)
err = spdeRes.makeReady(True)

### Projection Matrix: mesh to grid

In [ ]:
Aprojg = gl.ProjMatrix(gridExt, mesh).toTL()

### Simulation

In [ ]:
Q = spdeRes.getQ().toTL()
cholQ = cholesky(Q)

In [ ]:
u = np.random.normal(size = Q.shape[0])
gridExt["ManualSimu"] = cholQ.apply_Pt(cholQ.solve_Lt(1./np.sqrt(cholQ.D())*u))

gp.init(figsize=figsize,flagEqual=True)
gp.raster(gridExt, "ManualSimu", useSel=True)
gp.decoration(title="Simulation (Manual)")
print(f"Variance = {round(np.var(gridExt['ManualSimu'][np.where(gridExt['NewSel']==1)]),4)}")

### Kriging

In [ ]:
Aproj = spdeRes.getProj().toTL()
invnoise = spdeRes.getInvNoise().toTL()

Qop = Q + Aproj.T @ invnoise @ Aproj
cholQop =  cholesky(Qop)

kriging = cholQop.solve_A(Aproj.T @ (invnoise @ dat["SimuSPDE"]))
gridExt["ManualKrig"] = kriging

In [ ]:
ax = plt.scatter(gridExt["ManualKrig"][indSel],gridExt["*Chol.*estim"][indSel],s=1)
p = plt.plot([-3,3],[-3,3],c="r")

## Likelihood

Manually with Cholesky vs. matrix-free approach with SPDE api.

In [ ]:
def solveMat(cholQop,x):
    return cholQop.solve_A(x)

def invSigma(invnoise,Aproj,cholQop,x):
#    return 1./sigma2 * (x - 1./sigma2 * Aproj @ solveMat(cholQop, Aproj.T @ x))
    return invnoise @ x - invnoise @ Aproj @ solveMat(cholQop, Aproj.T @ (invnoise @ x)) 

def detQ(cholQ):
    return cholQ.logdet()

x = dat["SimuSPDE"]
ones = np.ones_like(x)
invSigmaOnes = invSigma(invnoise,Aproj,cholQop,ones)
mu  = np.sum(x * invSigmaOnes) / np.sum(ones * invSigmaOnes) 
print(f"Value for MU = {round(mu,4)}")

In [ ]:
spdeChol = gl.SPDE(dat, model, True, False, 1)
spdeChol.setMeshes(True, meshes)
err = spdeChol.makeReady(True)
invnoise = spdeChol.getInvNoise().toTL()

Q = spdeChol.getQ().toTL()
cholQ = cholesky(Q)

Aproj = spdeChol.getProj().toTL()

Qop = Q + Aproj.T @ invnoise @ Aproj
cholQop =  cholesky(Qop)

In [ ]:
a_quad = np.sum((x-mu)*invSigma(invnoise,Aproj,cholQop,x-mu))
b_quad = spdeChol.getSPDEOp().computeQuadratic(x - mu)
print(f"Quadratic (manual) = {round(a_quad,4)}")
print(f"Quadratic (spde)   = {round(b_quad,4)}")
print(f"-> Relative difference quadratic = {round(100*(b_quad - a_quad) / a_quad,2)}%")

In [ ]:
a_op = detQ(cholQop)
b_op = spdeChol.getSPDEOp().computeLogDetOp()
print(f"log_det_op (manual) = {round(a_op,4)}")
print(f"log_det_op (spde)   = {round(b_op,4)}")
print(f"-> Relative difference = {round(100*(b_op-a_op)/a_op, 2)}%")

In [ ]:
a_one = detQ(cholQ)
b_one = spdeChol.getPrecisionKrig().computeLogDet()
print(f"log_det_Q (manual) = {round(a_one,4)}")
print(f"log_det_Q (spde)   = {round(b_one,4)}")
print(f"-> Relative difference = {round(100*(b_one-a_one)/a_one,2)}%")

### Comparing the different outputs

- Manual calculation

In [ ]:
logdetnoise = len(x) * np.log(nugget)
logdet = a_op - a_one + logdetnoise
a = -0.5 * (a_quad + logdet + len(x) * np.log(2. * np.pi))
print("Likelihood calculation (manual):")
print(f"log_det_op      = {round(a_op,4)}")
print(f"log_det_Q       = {round(a_one,4)}")
print(f"log_det_Noise   = {round(logdetnoise,4)}")
print(f"log_determinant = {round(logdet,4)}")
print(f"Quadratic term  = {round(a_quad,4)}")
print(f"-> Likelihood (manual) = {round(a,4)}")

- Using the new API (we use the same 'mesh' as for the manual case to obtain the same results).

In [ ]:
useCholesky = 1
meshes = gl.VectorMeshes()
meshes.push_back(spdeChol.getMeshesK()[0])
b2 = gl.logLikelihoodSPDE(dat,model,useCholesky=useCholesky, meshes=meshes, params=params, verbose=True)
print(f"-> likelihood (api) cholesky=1 {round(b2,4)}")

In [ ]:
useCholesky = 0

spdeMF = gl.SPDE(dat, model, True, False, 0)
spdeMF.setMeshes(True, meshes)
err = spdeMF.makeReady(True)

In [ ]:
useCholesky = 0
nMC = 100
params = gl.SPDEParam.create(nMC=nMC)
b2 = gl.logLikelihoodSPDE(dat,model,useCholesky=useCholesky, meshes=meshes, params=params, verbose=True)
print(f"-> likelihood by New API with cholesky=0 {round(b2,4)}")